In [105]:
import pandas as pd
from sklearn.model_selection import train_test_split
from thinc.layers.padded2list import forward

In [106]:
df = pd.read_csv('housing.csv')
df.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


In [107]:
df.isnull().sum()

RM         0
LSTAT      0
PTRATIO    0
MEDV       0
dtype: int64

In [108]:
sum(df.duplicated())

0

In [109]:
df.columns = df.columns.str.lower()

In [110]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [112]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((391, 3), (98, 3), (391,), (98,))

In [113]:
type(X_train)

numpy.ndarray

In [114]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [115]:
import torch

In [116]:
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [117]:
type(X_train_tensor)

torch.Tensor

In [118]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((391, 3), (98, 3), torch.Size([391]), torch.Size([98]))

In [119]:
import torch
import torch.nn as nn
import torch.optim as optim

class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 1),
        )

    def forward(self, features):
        return self.network(features)

# Define the model
num_features = X_train_tensor.shape[1]
model = Model(num_features)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Use Adam for better optimization
loss_function = nn.MSELoss()  # For binary classification

# Training Loop
epochs = 250
for epoch in range(epochs):
    model.train()  # Set the model to training mode

    # Forward pass
    y_pred = model(X_train_tensor)
    
    # Compute loss
    loss = loss_function(y_pred.squeeze(), y_train)  # Squeeze for shape matching
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/250], Loss: 231878197248.0000
Epoch [2/250], Loss: 231878066176.0000
Epoch [3/250], Loss: 231877918720.0000
Epoch [4/250], Loss: 231877738496.0000
Epoch [5/250], Loss: 231877607424.0000
Epoch [6/250], Loss: 231877459968.0000
Epoch [7/250], Loss: 231877279744.0000
Epoch [8/250], Loss: 231877132288.0000
Epoch [9/250], Loss: 231876952064.0000
Epoch [10/250], Loss: 231876804608.0000
Epoch [11/250], Loss: 231876657152.0000
Epoch [12/250], Loss: 231876509696.0000
Epoch [13/250], Loss: 231876329472.0000
Epoch [14/250], Loss: 231876214784.0000
Epoch [15/250], Loss: 231876034560.0000
Epoch [16/250], Loss: 231875887104.0000
Epoch [17/250], Loss: 231875739648.0000
Epoch [18/250], Loss: 231875559424.0000
Epoch [19/250], Loss: 231875395584.0000
Epoch [20/250], Loss: 231875280896.0000
Epoch [21/250], Loss: 231875084288.0000
Epoch [22/250], Loss: 231874936832.0000
Epoch [23/250], Loss: 231874805760.0000
Epoch [24/250], Loss: 231874641920.0000
Epoch [25/250], Loss: 231874461696.0000
Epoch [26

In [120]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Testing phase (metrics evaluation)
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_test_pred = model(X_test_tensor).squeeze()  # Predicted values
    y_test_numpy = y_test.numpy()                # Actual test values (convert to numpy for metrics)
    y_test_pred_numpy = y_test_pred.numpy()

    # Calculate metrics
    mae = mean_absolute_error(y_test_numpy, y_test_pred_numpy)
    mse = mean_squared_error(y_test_numpy, y_test_pred_numpy)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_numpy, y_test_pred_numpy)

    print(f"Test Metrics:")
    print(f"  MAE:  {mae:.4f}")
    print(f"  MSE:  {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²:   {r2:.4f}")


Test Metrics:
  MAE:  461070.4375
  MSE:  240967368704.0000
  RMSE: 490884.2812
  R²:   -7.4853
